In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import joblib

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
df2 = pd.DataFrame()
n = [3, 4, 5, 12, 13, 14, 23, 24, 25, 44]
for i in range(1, 76):
    if i not in n:
        df = pd.read_excel("B:\GroundwaterRech\GithubRepo\Restart\Dataset\District Wise\\3\\" + str(i) + ".xlsx")
        df1 = df.drop([ 'Latitude','Longitude','Year'], axis=1)
        df2 = pd.concat([df2, df1])

In [ ]:
df2

In [ ]:
df2 = df2.reset_index()

In [ ]:
df2.drop('index', inplace=True, axis=1)

In [ ]:
df2

In [ ]:
# df2.to_csv('Feat.csv',index=False)

In [ ]:
df2['Soil Type'].unique()

In [ ]:
df2['LULC'].unique()

In [ ]:
df2['Soil Type'] = df2['Soil Type'].replace('Builtup', 1)
df2['Soil Type'] = df2['Soil Type'].replace('Bulitup', 1)

df2['Soil Type'] = df2['Soil Type'].replace('Haplic Luvisols', 2)
df2['Soil Type'] = df2['Soil Type'].replace('Haplic Calcisols', 3)
df2['Soil Type'] = df2['Soil Type'].replace('Loam', 4)
df2['Soil Type'] = df2['Soil Type'].replace('Calcaric Fluvisols', 5)


In [ ]:
X = df2.drop('Groundwater Recharge', axis=1)
y = df2['Groundwater Recharge']

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X

In [ ]:
scaler = MinMaxScaler()

In [ ]:
joblib.dump(scaler,"B:\GroundwaterRech\GithubRepo\Restart\Results\Pretrained Model\scaler(Pre-Post).pkl")

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# train_data1, valid_data, train_labels1, valid_labels = train_test_split(X, y, test_size=0.1, random_state=42)
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
train_data=scaler.fit_transform(train_data)
test_data=scaler.transform(test_data)

In [ ]:
train_data.shape

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)  #For cross validation

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import hydroeval as he
from sklearn.utils import resample

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
d = {}

In [ ]:
def calculate_nse(observed, predicted):
    observed = np.array(observed)
    predicted = np.array(predicted)

    mean_observed = np.mean(observed)

    sse = np.sum((observed - predicted) ** 2)

    sv = np.sum((observed - mean_observed) ** 2)

    nse = 1 - (sse / sv)

    return nse

In [ ]:
def hpt(model, params):
    grid_search = GridSearchCV(model, param_grid=params, n_jobs=-1, cv=kfold, scoring='neg_mean_squared_error',
                               return_train_score=True)
    grid_result = grid_search.fit(train_data, train_labels)
    print(f'Best Score: {grid_result.best_score_}\nParams: {grid_result.best_params_}')
    rcscore = grid_result.score(test_data, test_labels)
    print(f'Test_score: {rcscore}')

In [ ]:
def train_model(model, name):
    model.fit(train_data, train_labels)
    y_pred = model.predict(test_data)
    y_pred_train = model.predict(train_data)
    r2 = r2_score(test_labels, y_pred)
    ms = mean_squared_error(test_labels, y_pred)
    print(f'R2_score: {r2}, RMSE: {math.sqrt(ms)}')

    train_score = mean_squared_error(train_labels, y_pred_train)
    test_score = mean_squared_error(test_labels, y_pred)
    r2_s = r2_score(test_labels, y_pred)
    mae_test = mean_absolute_error(test_labels, y_pred)
    mae_train = mean_absolute_error(train_labels, y_pred_train)

    n1 = len(y_pred)
    p = X.shape[1]
    adj_r2 = 1 - (1 - r2_s) * (n1 - 1) / (n1 - p - 1)

    a = test_labels
    b = y_pred
    c2 = a.sub(b, fill_value=0)
    d2 = c2.pow(2)
    d_sum2 = d2.sum()
    a_mean2 = a.mean()
    e2 = ((a - a_mean2).pow(2).sum())
    f = 1 - (d_sum2 / e2)
    print(f'MAE_train: {mae_train}    MAE_test:{mae_test}     NSE2:{calculate_nse(a, b)}')
    print(f'MSE_train:  {train_score}  MSE_test: {test_score}')
    print(f'adjusted_R2:{adj_r2}')
    print(f'NSE:{he.nse(np.array(y_pred), np.array(test_labels))}')
    d[name] = (train_score, test_score, r2_s, mae_train, mae_test, f, adj_r2)
    # fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

    plt.figure(figsize=(8, 4))
    # sns.scatterplot(x=test_labels, y=y_pred)
    sns.scatterplot(x=test_labels, y=y_pred)
    coefficients = np.polyfit(test_labels, y_pred, 1)
    trendline_x = np.linspace(min(test_labels), max(test_labels), 100)
    trendline_y = np.polyval(coefficients, trendline_x)
    plt.plot(trendline_x, trendline_y, linestyle='dotted', color='red')
    plt.xlabel('True Groundwater Recharge (m)',fontsize=13,fontweight='bold')
    plt.ylabel('Predicted Groundwater Recharge (m)',fontsize=13,fontweight='bold')
    plt.title(f'Scatter Plot: {name}',fontsize=14,fontweight='bold')
    # plt.grid(False)
    plt.tight_layout()
    plt.savefig(f"B:\GroundwaterRech\GithubRepo\Restart\Results\Images(new)\Model Performance Plots\\{name}_Scatterplot.png",dpi=1200)
    plt.show()

    plt.figure(figsize=(7, 4))
    sns.lineplot(x=range(1, len(test_labels) + 1), y=test_labels, label='Actual')
    sns.lineplot(x=range(1, len(y_pred) + 1), y=y_pred, label='Predicted')
    plt.xlabel('Test Data (Years)',fontsize=13,fontweight='bold')
    plt.ylabel('Groundwater Recharge (m)',fontsize=13,fontweight='bold')
    plt.title(f'Line Plot: {name}',fontsize=14,fontweight='bold')
    plt.legend()
    # plt.grid(False)

    plt.tight_layout()

    # Display the line plot

    plt.savefig(f"B:\GroundwaterRech\GithubRepo\Restart\Results\Images(new)\Model Performance Plots\\{name}_Lineplot.png",dpi=1200)
    plt.show()
    joblib.dump(model,"B:\GroundwaterRech\GithubRepo\Restart\Results\Pretrained Model\Pre-post\\"+str(name)+".pkl")

In [ ]:
#Random Forest
train_model(RandomForestRegressor(max_depth=14, max_features=19, random_state=45), "Random Forest Regressor")

In [ ]:
hpt(RandomForestRegressor(), {'max_depth': [i for i in range(8, 15)], 'max_features': [i for i in range(2, 20)]})

In [ ]:
#Decision Tree
train_model(DecisionTreeRegressor(max_depth=14, max_features=19, random_state=45), "Decision Tree Regressor")

In [ ]:
hpt(DecisionTreeRegressor(), {'max_depth': [i for i in range(8, 15)], 'max_features': [i for i in range(2, 20)]})

In [ ]:
#GBR
train_model(GradientBoostingRegressor(max_depth=7, max_features=14, n_estimators=100, random_state=10),
            name='Gradient Boosting Regressor')

In [ ]:
hpt(GradientBoostingRegressor(), {'max_depth': [i for i in range(8, 15)], 'max_features': [i for i in range(10, 20)],
                                  'n_estimators': [100, 150, 500]})

In [ ]:
import xgboost

train_model(xgboost.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=150, random_state=45), 'XG Boost')

Uncertainity Analysis

In [ ]:
n_bootstrap = 1000
bootstrap_predictions = np.zeros((n_bootstrap, len(test_labels)))
for i in range(n_bootstrap):
    # Resample the training data with replacement
    X_boot, y_boot = resample(train_data, train_labels, random_state=i)
    # Fit XGB model to the bootstrapped data
    xgb_boot = xgboost.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=150, random_state=i)
    xgb_boot.fit(X_boot, y_boot)

    # Make predictions on the test set
    bootstrap_predictions[i] = xgb_boot.predict(test_data)

In [ ]:
mean_predictions = np.mean(bootstrap_predictions, axis=0)
std_predictions = np.std(bootstrap_predictions, axis=0)
mse_mean = mean_squared_error(test_labels, mean_predictions)
r2_mean=r2_score(test_labels,mean_predictions)
mean_mae=mean_absolute_error(test_labels,mean_predictions)

In [ ]:
cv = std_predictions / mean_predictions
mean_cv = np.mean(cv)

In [ ]:
print(f"Mean Squared Error: {mse_mean}\nMAE: {mean_mae}\nr2: {r2_mean}\nCV: {mean_cv}")

In [ ]:
confidence_interval = np.percentile(bootstrap_predictions, [2.5, 97.5], axis=0)

In [ ]:
plt.figure(figsize=(10, 6))
plt.fill_between(range(len(test_labels[:100])), confidence_interval[0][:100], confidence_interval[1][:100], color='gray', alpha=0.4, label='95% Confidence Interval')

plt.plot(range(len(test_labels[:100])), test_labels[:100], label='True Values')
plt.plot(range(len(test_labels[:100])), mean_predictions[:100], label='Mean Predictions', color='red')

plt.title('Uncertainty Analysis on Test Data ',fontsize=16,fontweight='bold')
plt.xlabel('Sample Index',fontsize=14,fontweight='bold')
plt.ylabel('Groundwater Recharge (m)',fontsize=14,fontweight='bold')
plt.legend()
plt.grid(True)
plt.savefig('B:\GroundwaterRech\GithubRepo\Restart\Results\Images\\uncertainty1(pre-post).png', dpi=1200)
plt.show()

In [ ]:

m=joblib.load("B:\GroundwaterRech\GithubRepo\Restart\Results\Pretrained Model\\"+str("XG Boost")+".pkl")

In [ ]:
m.feature_importances_

In [ ]:
hpt(xgboost.XGBRegressor(),
    {'max_depth': [i for i in range(10, 15)], 'n_estimators': [100, 150, 250, 400],
     'learning_rate': [0.1, 0.01, 0.001], })

In [ ]:
import lightgbm as lgb

train_model(lgb.LGBMRegressor(max_depth=13, n_estimators=800, learning_rate=0.1, colsample_bytree=0.9, subsample=0.8,
                              random_state=45), 'LGBM Regressor')

In [ ]:
hpt(lgb.LGBMRegressor(),
    {'max_depth': [i for i in range(10, 15)], 'n_estimators': [100, 500, 1000], 'learning_rate': [0.1, 0.01, 0.001],
     'subsample': [0.8, 0.9, 1.0],
     'colsample_bytree': [0.8, 0.9, 1.0]})

In [ ]:
#Linear Regression
train_model(LinearRegression(), "Linear Regression")

In [ ]:
#Ridge Regression
train_model(Ridge(alpha=10), 'Ridge Regression')

In [ ]:
hpt(Ridge(), {'alpha': [0.1, 1, 10, 100, 0.01]})

Polynomial Regression

In [ ]:
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(train_data)
x_test_poly = poly.fit_transform(test_data)

plr = LinearRegression()
plr.fit(x_train_poly, train_labels)
y_pred = plr.predict(x_test_poly)
y_pred_train = plr.predict(x_train_poly)
r2 = r2_score(test_labels, y_pred)
ms = mean_squared_error(test_labels, y_pred)
print(r2, math.sqrt(ms))

train_score = mean_squared_error(train_labels, y_pred_train)
test_score = mean_squared_error(test_labels, y_pred)
r2_s = r2_score(test_labels, y_pred)
mae_test = mean_absolute_error(test_labels, y_pred)
mae_train = mean_absolute_error(train_labels, y_pred_train)

a = test_labels
b = y_pred
c2 = a.sub(b, fill_value=0)
d2 = c2.pow(2)
d_sum2 = d2.sum()
a_mean2 = a.mean()
e2 = ((a - a_mean2).pow(2).sum())
f = 1 - (d_sum2 / e2)

n1 = len(y_pred)
p = X.shape[1]
adj_r2 = 1 - (1 - r2_s) * (n1 - 1) / (n1 - p - 1)
d['Polynomial Regression'] = (train_score, test_score, r2_s, mae_train, mae_test, f,adj_r2)

plt.figure(figsize=(8, 4))
# sns.scatterplot(x=test_labels, y=y_pred)
sns.scatterplot(x=test_labels, y=y_pred)
coefficients = np.polyfit(test_labels, y_pred, 1)
trendline_x = np.linspace(min(test_labels), max(test_labels), 100)
trendline_y = np.polyval(coefficients, trendline_x)
plt.plot(trendline_x, trendline_y, linestyle='dotted', color='red')
plt.xlabel('True Groundwater Recharge (m)',fontsize=13,fontweight='bold')
plt.ylabel('Predicted Groundwater Recharge (m)',fontsize=13,fontweight='bold')
plt.title(f'Scatter Plot: Polynomial Regression(degree=3)',fontsize=14,fontweight='bold')
# plt.grid(False)
plt.tight_layout()
plt.savefig(f"B:\GroundwaterRech\GithubRepo\Restart\Results\Images(new)\Model Performance Plots\PolynomialReg_Scatterplot.png",dpi=1200)
plt.show()

plt.figure(figsize=(8, 4))
sns.lineplot(x=range(1, len(test_labels) + 1), y=test_labels, label='Actual')
sns.lineplot(x=range(1, len(y_pred) + 1), y=y_pred, label='Predicted')
plt.xlabel('Test Data (Years)',fontsize=13,fontweight='bold')
plt.ylabel('Groundwater Recharge (m)',fontsize=13,fontweight='bold')
plt.title(f'Line Plot: Polynomial Regression(degree=3) ',fontsize=14,fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig(f"B:\GroundwaterRech\GithubRepo\Restart\Results\Images(new)\Model Performance Plots\PolynomialReg_Lineplot.png",dpi=1200)

# Display the line plot
plt.show()

In [ ]:
d

ANN

In [ ]:
import tensorflow as tf

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(25, input_shape=(27,), activation='relu'),
    # tf.keras.layers.Dropout()
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(train_data, train_labels, epochs=150, batch_size=32)

In [ ]:
# Evaluate the model on the test set
test_score = model.evaluate(test_data, test_labels)
train_score = model.evaluate(train_data, train_labels)
y_pred = model.predict(test_data)
y_pred_train = model.predict(train_data)
print(f'Mean squared error on test set: {test_score} train set: {train_score}')
print("R2 square", r2_score(test_labels, y_pred))

In [ ]:
train_score = mean_squared_error(train_labels, np.squeeze(y_pred_train))
test_score = mean_squared_error(test_labels, np.squeeze(y_pred))
r2_s = r2_score(test_labels, np.squeeze(y_pred))
mae_test = mean_absolute_error(test_labels, np.squeeze(y_pred))
mae_train = mean_absolute_error(train_labels, np.squeeze(y_pred_train))

n1 = len(y_pred)
p = X.shape[1]
adj_r2 = 1 - (1 - r2_s) * (n1 - 1) / (n1 - p - 1)
a, b = list(test_labels), list(y_pred)
n = 0
d1 = 0
d_mean = test_labels.mean()
for i in range(len(y_pred)):
    n = n + math.pow((a[i] - b[i]), 2)
    d1 = d1 + math.pow((a[i] - d_mean), 2)
nse = 1 - (n / d1)



d['MLP'] = (train_score, test_score, r2_s, mae_train, mae_test, nse,adj_r2)


plt.figure(figsize=(8, 4))
# sns.scatterplot(x=test_labels, y=y_pred)
sns.scatterplot(x=test_labels, y=np.squeeze(y_pred))
coefficients = np.polyfit(test_labels, np.squeeze(y_pred), 1)
trendline_x = np.linspace(min(test_labels), max(test_labels), 100)
trendline_y = np.polyval(coefficients, trendline_x)
plt.plot(trendline_x, trendline_y, linestyle='dotted', color='red')
plt.xlabel('True Groundwater Recharge (m)',fontsize=13,fontweight='bold')
plt.ylabel('Predicted Groundwater Recharge (m)',fontsize=13,fontweight='bold')
plt.title(f'Scatter Plot: Multi Layer Perceptron(MLP)',fontsize=14,fontweight='bold')
# plt.grid(False)
plt.tight_layout()
plt.savefig(f"B:\GroundwaterRech\GithubRepo\Restart\Results\Images(new)\Model Performance Plots\MLP_Scatterplot.png",dpi=1200)

plt.show()

plt.figure(figsize=(8, 4))
sns.lineplot(x=range(1, len(test_labels) + 1), y=test_labels, label='Actual')
sns.lineplot(x=range(1, len(np.squeeze(y_pred)) + 1), y=np.squeeze(y_pred), label='Predicted')
plt.xlabel('Test Data (Years)',fontsize=13,fontweight='bold')
plt.ylabel('Groundwater Recharge (m)',fontsize=13,fontweight='bold')
plt.title(f'Line Plot: Multi Layer Perceptron(MLP) ',fontsize=14,fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig(f"B:\GroundwaterRech\GithubRepo\Restart\Results\Images(new)\Model Performance Plots\MLP_Lineplot.png",dpi=1200)

# Display the line plot
plt.show()

In [ ]:
d

In [ ]:
df = pd.DataFrame.from_dict(d, orient='index',
                            columns=['Train MSE', 'Test MSE', 'R2_score', 'Train MAE', 'Test MAE', 'NSE','Adj_r2'])
df.to_excel('Models_Results(Districts).xlsx')